## Config LLM + Embedding model

In [1]:
import autogen
from memgpt.autogen.memgpt_agent import create_memgpt_autogen_agent_from_config
from autogen.agentchat.contrib.retrieve_assistant_agent import RetrieveAssistantAgent
from autogen.agentchat.contrib.retrieve_user_proxy_agent import RetrieveUserProxyAgent
from autogen import UserProxyAgent, config_list_from_json
from autogen.agentchat.contrib.teachable_agent import TeachableAgent
import os
import random
random_seed = random.randint(0, 1000)

config_list = [
    {
        #"model": "zephyr-7B-beta",
        "model":"dolphin-2.1-mistral-7b",
        #"api_base": "http://172.19.208.1:1300/v1",
        "api_base": "http://localhost:5001/v1",
        "request_timeout":600,
        "timeout": 120,

    }
]

code_execution_config={"work_dir": "coding",
                       "use_docker":"python:3"}

llm_config = {"config_list": config_list,
              #"context":"",
              #"prompt":"{problem} Solve the problem and explain the reasoning step by step",
              "use_cache":True,
              "seed": 1,
              "temperature":0
              }

from chromadb.utils import embedding_functions
#ef = embedding_functions.SentenceTransformerEmbeddingFunction(model_name="all-MiniLM-L6-v2") 
ef = embedding_functions.InstructorEmbeddingFunction(model_name="hkunlp/instructor-large", device="cuda")




/home/mohcine/mambaforge/envs/autogen/lib/python3.10/site-packages/InstructorEmbedding/instructor.py:7: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import trange


load INSTRUCTOR_Transformer
max_seq_length  512


In [7]:
import PyPDF2
import re

# Define the paths to your two different resume PDF files
resume_file_1 = "../cv.pdf"
resume_file_2 = "../cv_eng_long.pdf"

# Function to parse a resume PDF
# Function to extract text from a PDF
def extract_text_from_pdf(pdf_file):
    text = ""
    with open(pdf_file, "rb") as file:
        pdf_reader = PyPDF2.PdfFileReader(file)
        for page_num in range(pdf_reader.getNumPages()):
            page = pdf_reader.getPage(page_num)
            text += page.extractText()
    return text

# Call the function and store the extracted text
extracted_text_1 = extract_text_from_pdf(resume_file_1)
extracted_text_2 = extract_text_from_pdf(resume_file_1)

# Print or use the extracted text as needed
print(extracted_text_1)
print(extracted_text_2)

with open("raw_resume.txt", 'w') as f:
    # Write the parsed HTML code to the file
    f.write(str(extracted_text_1+extracted_text_2))


Mohcine
EL HARRAS
Data Scien tist
Looking f or a posit on as S enior  Data Scientist
Education
Centralesupélec - U niversité Paris-S aclay
Grande E cole o f Engineering France
Machine L earning
Comput er vision
Software engineering
Econom y / Financ e
Lycée G eorges Cabanis
Prepar atory classes f or engineering schools ( CPGE) Brive la
gaillar de, F rance
Awarded with H igh H ounour s. Ranked 16th/1250  (Top 1.3% )
El farabi H igh S chool
Scien tiﬁc B accalaur eate Salé, M orocco
Ranked 30th  in the c ountry and 3rd in m y region. H igh H ounour s.
Work experience
EDF R&D
Machine learning engineer  Palaiseau, F rance
Resear ch E ngineer  (from 10/2021 t o 01/2024)
Comput er vision on lo w voltag e grid equipmen t (CIRED 2023).
LLMs + Q Aretrieval with embeddings t o build a Q A conversational
agent with in tern documen ts.
Anomaly  detection and r eliability  study  on BESS s (Battery
manag emen t systems)
Digital twin o f the in forma tion s ystem o f EDF
Resear ch E ngineering Appren

In [4]:
pdf_text

NameError: name 'pdf_text' is not defined

## Load documents

In [2]:
from autogen.retrieve_utils import get_file_from_url,get_files_from_dir, split_files_to_chunks, create_vector_db_from_dir, query_vector_db
import requests
from bs4 import BeautifulSoup
from langchain.document_loaders import BSHTMLLoader

urls = ["https://uca.edu/politicalscience/home/research-projects/dadm-project/middle-eastnorth-africapersian-gulf-region/israelpalestine-1948-present",
        "https://news.un.org/fr/story/2023/10/1140027",
        "https://www.amnesty.org/fr/location/middle-east-and-north-africa/israel-and-occupied-palestinian-territories/report-israel-and-occupied-palestinian-territories",
        "https://www.unicef.fr/article/israel-palestine-les-enfants-paient-le-prix-de-la-guerre",
        
        ]

for url in urls:
    # Download the HTML file from the URL
    response = requests.get(url)
    # Parse the downloaded content using BeautifulSoup
    soup = BeautifulSoup(response.content, "html.parser")
    # Create or open the output file in write mode
    output_path_html = 'data_html/'+url.split("/")[-1]+'.html'
    with open(output_path_html, 'w') as f:
        # Write the parsed HTML code to the file
        f.write(str(soup))
    output_path_txt = 'data/'+url.split("/")[-1]+'.txt'
    soup_processed = BSHTMLLoader(output_path_html).load()
    with open(output_path_txt, 'w') as f:
        # Write the processed HTML code to a txt file
        f.write(str(soup_processed))




#split_files_to_chunks(files = get_files_from_dir("data/"))
#get_file_from_url(output_path)
#split_files_to_chunks(files = get_files_from_dir("data/"))


In [3]:
create_vector_db_from_dir(dir_path="data/",
                          db_path = "./data_emb",
                          collection_name = "all-my-documents",
                          embedding_function = ef)

max_tokens is too small to fit a single line of text. Breaking this line:
	[Document(page_content='\n\n\n31.  Israel/Palestine (1948-present) \n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\ ...
Failed to split docs with must_break_at_empty_line being True, set to False.
max_tokens is too small to fit a single line of text. Breaking this line:
	[Document(page_content='\n\n\n31.  Israel/Palestine (1948-present) \n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\ ...
max_tokens is too small to fit a single line of text. Breaking this line:
	l 2, 2001.\xa0 EU Commissioner for External Relations Chris Patton appealed for an immediate ceasefi ...
max_tokens is too small to fit a single line of text. Breaking this line:
	[Document(page_content="\n\n\n\n\n\n\n\n\n\n\n\nIsraël-Palestine\xa0: la tragédie des enfants - UNIC ...
Failed to split docs with must_break_at_empty_line being True, set to False.
max_tokens is too small to fit a single line of text. Breaking this line:
	[Document(page_content="\n\n\n\n\n\n\n\n\n\n\n\nIsr

In [5]:
query_result = query_vector_db(query_texts=["what is memgpt"],
                n_results=2,
                db_path = "./data_emb",
                collection_name = "all-my-documents",
                embedding_function = ef)

In [11]:
print(len(query_result["documents"][0]))
query_result["documents"]

2


[['Preprint\nMEMGPT: T OWARDS LLM S AS OPERATING SYSTEMS\nCharles Packer Vivian Fang Shishir G. Patil\nKevin Lin Sarah Wooders Joseph E. Gonzalez\nUC Berkeley\nhttps://memgpt.ai\nABSTRACT\nLarge language models (LLMs) have revolutionized AI, but are constrained by\nlimited context windows, hindering their utility in tasks like extended conversa-\ntions and document analysis. To enable using context beyond limited context win-\ndows, we propose virtual context management, a technique drawing inspiration\nfrom hierarchical memory systems in traditional operating systems that provide\nthe appearance of large memory resources through data movement between fast\nand slow memory. Using this technique, we introduce MemGPT (Memory-GPT),\na system that intelligently manages different memory tiers in order to effectively\nprovide extended context within the LLM’s limited context window, and utilizes\ninterrupts to manage control flow between itself and the user. We evaluate our\nOS-inspired desi

In [13]:
query_result["documents"][0][0]

'Preprint\nMEMGPT: T OWARDS LLM S AS OPERATING SYSTEMS\nCharles Packer Vivian Fang Shishir G. Patil\nKevin Lin Sarah Wooders Joseph E. Gonzalez\nUC Berkeley\nhttps://memgpt.ai\nABSTRACT\nLarge language models (LLMs) have revolutionized AI, but are constrained by\nlimited context windows, hindering their utility in tasks like extended conversa-\ntions and document analysis. To enable using context beyond limited context win-\ndows, we propose virtual context management, a technique drawing inspiration\nfrom hierarchical memory systems in traditional operating systems that provide\nthe appearance of large memory resources through data movement between fast\nand slow memory. Using this technique, we introduce MemGPT (Memory-GPT),\na system that intelligently manages different memory tiers in order to effectively\nprovide extended context within the LLM’s limited context window, and utilizes\ninterrupts to manage control flow between itself and the user. We evaluate our\nOS-inspired design

## Define agents retrieval

In [2]:
assistant = RetrieveAssistantAgent(
    name="assistant",
    system_message="You are a helpful assistant.",
    llm_config=llm_config,
)

ragproxyagent = RetrieveUserProxyAgent(
    name="ragproxyagent",
    retrieve_config={
        "task": "qa",
        "docs_path": "/home/mohcine/work/personal/autogen_memgpt/data/",
        "embedding_function": ef
    },
)


In [3]:
assistant.reset()
ragproxyagent.initiate_chat(assistant, problem="what is memgpt? ?")


Trying to create collection.


max_tokens is too small to fit a single line of text. Breaking this line:
	[Document(page_content='\n\n\n31.  Israel/Palestine (1948-present) \n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\ ...
Failed to split docs with must_break_at_empty_line being True, set to False.
max_tokens is too small to fit a single line of text. Breaking this line:
	[Document(page_content='\n\n\n31.  Israel/Palestine (1948-present) \n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\ ...
max_tokens is too small to fit a single line of text. Breaking this line:
	 violence between September 1993 and September 2000.\nCrisis Phase (September 28, 2000-February 8, 2 ...
max_tokens is too small to fit a single line of text. Breaking this line:
	[Document(page_content="\n\n\n\n\n\n\n\n\n\n\n\nIsraël-Palestine\xa0: la tragédie des enfants - UNIC ...
Failed to split docs with must_break_at_empty_line being True, set to False.
max_tokens is too small to fit a single line of text. Breaking this line:
	[Document(page_content="\n\n\n\n\n\n\n\n\n\n\n\nIsr

doc_ids:  [['doc_0', 'doc_2', 'doc_3', 'doc_1', 'doc_4', 'doc_6', 'doc_7', 'doc_5']]
Adding doc_id doc_0 to context.
Adding doc_id doc_2 to context.
Adding doc_id doc_3 to context.
ragproxyagent (to assistant):

You're a retrieve augmented chatbot. You answer user's questions based on your own knowledge and the
context provided by the user.
If you can't answer the question with or without the current context, you should reply exactly `UPDATE CONTEXT`.
You must give as short an answer as possible.

User's question is: what is memgpt? ?

Context is: [Document(page_content='\n403 Forbidden\n\n403 Forbidden\nnginx\n\n\n', metadata={'source': 'data_html/report-israel-and-occupied-palestinian-territories.html', 'title': '403 Forbidden'})]
 violence between September 1993 and September 2000.\nCrisis Phase (September 28, 2000-February 8, 2005): Ariel Sharon, right-wing opposition leader, visited the Temple Mount (Haram as-Sharif) in Jerusalem on September 28, 2000, resulting in the second Pale

## Code challenge

In [2]:
from autogen import AssistantAgent, UserProxyAgent, config_list_from_json
config_list = [
    {
        "model":"dolphin-2.1-mistral-7b",
        "api_base": "http://localhost:5001/v1",
        "request_timeout":600,
        "timeout": 120,
    }
]

# Load LLM inference endpoints from an env variable or a file
# See https://microsoft.github.io/autogen/docs/FAQ#set-your-api-endpoints
# and OAI_CONFIG_LIST_sample.json
assistant = AssistantAgent("assistant", llm_config={"config_list": config_list})
user_proxy = UserProxyAgent("user_proxy", code_execution_config={"work_dir": "coding","use_docker":False})
user_proxy.initiate_chat(assistant, message="Plot a chart of NVDA and TESLA stock price change YTD.")
# This initiates an automated chat between the two agents to solve the task


user_proxy (to assistant):

Plot a chart of NVDA and TESLA stock price change YTD.

--------------------------------------------------------------------------------
assistant (to user_proxy):

```bash
#!/bin/bash
# filename: nvda-tesla-ytd.sh

# Import necessary packages for data retrieval and plotting
pip install pandas yfinance matplotlib

# Set the tickers of NVDA (NVIDIA) and TESLA (Tesla Inc.)
nvda_ticker="NVDA"
tesla_ticker="TESLA"

# Retrieve daily stock prices for both NVDA and TESLA from Yahoo Finance using yfinance package for python
yf import stock as stk
nvda = stk.get_data(symbol=nvda_ticker, start='2021-01-01', progress=False)
tesla = stk.get_data(symbol=tesla_ticker, start='2021-01-01', progress=False)

# Calculate the YTD percentage change in prices for both NVDA and TESLA
nvda["YTD % Change"] = ((nvda['Close'] - nvda['Open']) / nvda['Open']) * 100
tesla["YTD % Change"] = ((tesla['Close'] - tesla['Open']) / tesla['Open']) * 100

# Plot a line chart using matplotlib for 

KeyboardInterrupt: 

## AutoGen's TeachableAgent

In [1]:
from autogen import UserProxyAgent, config_list_from_json
from autogen.agentchat.contrib.teachable_agent import TeachableAgent

In [2]:
config_list = [
    {
        "model":"dolphin-2.1-mistral-7b",
        "api_base": "http://localhost:5001/v1",
        "request_timeout":600,
        

    }
]

code_execution_config={"work_dir": "teached",
                       "use_docker":False}

llm_config = {"config_list": config_list,
              #"context":"",
              #"prompt":"{problem} Solve the problem and explain the reasoning step by step",
              "use_cache":False,
              "seed": 2,
              "temperature":0,
              "timeout": 120
              }



In [3]:
teachable_agent = TeachableAgent(
    name="teachableagent",
    llm_config=llm_config,
    teach_config={
        "reset_db": False,  # Use True to force-reset the memo DB, and False to use an existing DB.
        "path_to_db_dir": "./vb_data_agent"  # Can be any path.
    }
)

user = UserProxyAgent("user", human_input_mode="ALWAYS")

In [5]:
# This function will return once the user types 'exit'.
teachable_agent.initiate_chat(user, message="Hi, I'm a teachable user assistant! What's on your mind?")


teachableagent (to user):

Hi, I'm a teachable user assistant! What's on your mind?

--------------------------------------------------------------------------------
user (to teachableagent):

learn everything about this paper : https://arxiv.org/pdf/2310.08560v1.pdf

--------------------------------------------------------------------------------
teachableagent (to user):

Sure, I can help you learn more about the paper "Learning to Learn with Meta-Reinforcement Learning" by Zhang et al., which is available at https://arxiv.org/pdf/2310.08560v1.pdf. The paper discusses a novel approach called meta-reinforcement learning (MRL) that enables agents to learn new tasks more efficiently and adaptively by leveraging their previous experiences.

The main contributions of this work are:

1. Introducing MRL as an extension of reinforcement learning, which allows agents to transfer knowledge from one task to another in a meta-learning setting.
2. Proposing a new algorithm called Meta-Reinforceme

## Retrieval

In [6]:
import autogen
from memgpt.autogen.memgpt_agent import create_memgpt_autogen_agent_from_config
from autogen.agentchat.contrib.retrieve_assistant_agent import RetrieveAssistantAgent
from autogen.agentchat.contrib.retrieve_user_proxy_agent import RetrieveUserProxyAgent

assistant = RetrieveAssistantAgent(
    name="assistant",
    system_message="You are a helpful assistant.",
    llm_config=llm_config,
)

ragproxyagent = RetrieveUserProxyAgent(
    name="ragproxyagent",
    retrieve_config={
        "task": "qa",
        "docs_path": "data/2310.08560v1_2.pdf",
    },
)


In [8]:
get_files_from_dir("data/")


NameError: name 'get_files_from_dir' is not defined

In [7]:
assistant.reset()
ragproxyagent.initiate_chat(assistant, problem="What is memgpt?")


Trying to create collection.


max_tokens is too small to fit a single line of text. Breaking this line:
	Preprint ...
Failed to split docs with must_break_at_empty_line being True, set to False.
Number of requested results 20 is greater than number of elements in index 5, updating n_results = 5


doc_ids:  [['doc_2', 'doc_1', 'doc_0', 'doc_3', 'doc_4']]
Adding doc_id doc_2 to context.
Adding doc_id doc_1 to context.
ragproxyagent (to assistant):

You're a retrieve augmented chatbot. You answer user's questions based on your own knowledge and the
context provided by the user.
If you can't answer the question with or without the current context, you should reply exactly `UPDATE CONTEXT`.
You must give as short an answer as possible.

User's question is: What is memgpt?

Context is: of relying on recursive summaries to extend context, MemGPT is able to query past conversation
history in its Recall Memory to answer the DMR questions.
3.1.3 C ONVERSATION OPENER TASK (ENGAGEMENT )
In the ‘conversation opener’ task we evaluate an agent’s ability to craft engaging messages to the user
that draw from knowledge accumulated in prior conversations. To evaluate the ‘engagingness’ of a
conversation opener using the MSC dataset, we compare the generated opener to the gold personas:
an engagin